In [ ]:
!pip install scikit-plot
!pip install jax-unirep

Try to deal with GPU memory 
accroading to https://jax.readthedocs.io/en/latest/gpu_memory_allocation.html

In [1]:
import jax
# Global flag to set a specific platform, must be used at startup.
jax.config.update('jax_platform_name', 'cpu')

import os
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE']='False'
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION']='.95'
os.environ['XLA_PYTHON_CLIENT_ALLOCATOR']='platform'

import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt

from Bio import SeqIO
from jax_unirep import get_reps
from jax_unirep import evotune, fit
from jax_unirep.utils import dump_params


In [2]:
import gc
import jax 
def clear_jax_caches():
  """Utility to clear all the function caches in jax."""
  # main jit/pmap lu wrapped function caches - have to grab from closures
  jax.xla._xla_callable.__closure__[1].cell_contents.clear()
  jax.pxla.parallel_callable.__closure__[1].cell_contents.clear()
  # primitive callable caches
  jax.xla.xla_primitive_callable.cache_clear()
  jax.xla.primitive_computation.cache_clear()
  # jaxpr caches for control flow and reductions
  jax.lax.lax_control_flow._initial_style_jaxpr.cache_clear()
  jax.lax.lax_control_flow._fori_body_fun.cache_clear()
  jax.lax.lax._reduction_jaxpr.cache_clear()
  # these are trivial and only included for completeness sake
  jax.lax.lax.broadcast_shapes.cache_clear()
  jax.xla.xb.get_backend.cache_clear()
  jax.xla.xb.dtype_to_etype.cache_clear()
  jax.xla.xb.supported_numpy_dtypes.cache_clear()
    
def reset_device_memory(delete_objs=True):
    """Free all tracked DeviceArray memory and delete objects.

  Args:
    delete_objs: bool: whether to delete all live DeviceValues or just free.

  Returns:
    number of DeviceArrays that were manually freed.
  """
    dvals = (x for x in gc.get_objects() if isinstance(x, jax.xla.DeviceArray))
    n_deleted = 0
    for dv in dvals:
    
        if not isinstance(dv, jax.xla.DeviceConstant):
            try: 
                dv._check_if_deleted()  # pylint: disable=protected-access
                dv.device_buffer.delete()
                n_deleted += 1
            except:
                pass
        if delete_objs:

            del dv
    del dvals
    gc.collect()
    return n_deleted

In [2]:
fastas = "/home/kongkitimanonk/SCRATCH_NOBAK/cd-hit/bk_fasta/ERR2130640.assembly.len15.fasta"
plk = "/home/kongkitimanonk/SCRATCH_NOBAK/phase3/Rat.len15.pkl"

In [3]:
with open(fastas) as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    seqs = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        # Remove leading and trailing characters from a string
        seqs.append(str(seq_record.seq.strip('*')))
        lengths.append(len(seq_record.seq))

In [4]:
# dictionary of lists  
dict = {'ID': identifiers, 'Sequence': seqs, 'length': lengths}  
df = pd.DataFrame(dict) 
#df["Sequence"] =  seqs
#df.to_pickle(plk)    
df  

,ID,Sequence,length
0,0,FFFLDDFYCRCTEGEEEGRGKDRV,24
1,1,EANYVHGPFLFPLPPPPCIYNKNHLKKKKKK,31
2,2,VRTRGILKGSWDLPFPI,17
3,3,TKEVTINADTTCGNDWVCEHRWRQIRNMVAFRNEV,35
4,4,LQKVKFIWYNNVINPTL,17
...,...,...,...
311702,311702,CGHGQGCSLPGPIWGWGG,18
311703,311703,KGFGWVQVSPPNENVVV,17
311704,311704,KSQEATDHWHITAAHSS,17
311705,311705,GWKSRGGWDHRTPRVPR,17


Remove Duplicate

In [5]:
df.drop_duplicates(subset=['Sequence'],inplace=True)
df

,ID,Sequence,length
0,0,FFFLDDFYCRCTEGEEEGRGKDRV,24
1,1,EANYVHGPFLFPLPPPPCIYNKNHLKKKKKK,31
2,2,VRTRGILKGSWDLPFPI,17
3,3,TKEVTINADTTCGNDWVCEHRWRQIRNMVAFRNEV,35
4,4,LQKVKFIWYNNVINPTL,17
...,...,...,...
311702,311702,CGHGQGCSLPGPIWGWGG,18
311703,311703,KGFGWVQVSPPNENVVV,17
311704,311704,KSQEATDHWHITAAHSS,17
311705,311705,GWKSRGGWDHRTPRVPR,17


In [6]:
_h_avg, h_final, c_final= get_reps(df['Sequence'].to_list())
df['reps']=_h_avg.tolist()
df

,ID,Sequence,length,reps
0,0,FFFLDDFYCRCTEGEEEGRGKDRV,24,"[0.02810720168054104, -0.002855583792552352, 0..."
1,1,EANYVHGPFLFPLPPPPCIYNKNHLKKKKKK,31,"[0.04690786451101303, -0.10334441810846329, -0..."
2,2,VRTRGILKGSWDLPFPI,17,"[0.03444882109761238, -0.04888810217380524, -0..."
3,3,TKEVTINADTTCGNDWVCEHRWRQIRNMVAFRNEV,35,"[0.01922222599387169, -0.07419043034315109, 0...."
4,4,LQKVKFIWYNNVINPTL,17,"[0.03477690368890762, -0.02958511933684349, -0..."
...,...,...,...,...
311702,311702,CGHGQGCSLPGPIWGWGG,18,"[0.031068434938788414, -0.09170962125062943, -..."
311703,311703,KGFGWVQVSPPNENVVV,17,"[0.04786856472492218, -0.01857958547770977, -0..."
311704,311704,KSQEATDHWHITAAHSS,17,"[0.03583816811442375, -0.058845095336437225, -..."
311705,311705,GWKSRGGWDHRTPRVPR,17,"[0.03462590277194977, -0.05176414176821709, -0..."


In [7]:
df.to_pickle(plk)


## CD-100

In [9]:
fastas = "/home/ubuntu/data/bk_fasta/ERR2130640.assembly.len15.cd100.fasta"
with open(fastas) as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    seqs = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        # Remove leading and trailing characters from a string
        seqs.append(str(seq_record.seq.strip('*')))
        lengths.append(len(seq_record.seq))
# dictionary of lists  
dict = {'ID': identifiers, 'Sequence': seqs, 'length': lengths}  
df = pd.DataFrame(dict) 
#df["Sequence"] =  seqs
#df.to_pickle(plk)    
df  

,ID,Sequence,length
0,0,FFFLDDFYCRCTEGEEEGRGKDRV,24
1,1,EANYVHGPFLFPLPPPPCIYNKNHLKKKKKK,31
2,2,VRTRGILKGSWDLPFPI,17
3,3,TKEVTINADTTCGNDWVCEHRWRQIRNMVAFRNEV,35
4,5,GYGCAQRGKPGVYTKVCKYVNWIQQTIAGN,31
...,...,...,...
208414,311701,KLLSYCHENFIHVHCSLGR,19
208415,311702,CGHGQGCSLPGPIWGWGG,18
208416,311703,KGFGWVQVSPPNENVVV,17
208417,311705,GWKSRGGWDHRTPRVPR,17


In [10]:
# Setting my chunk size
chunk_size = 10000
# Assigning chunk numbers to rows
df['chunk'] = df['ID'].apply(lambda x: int(int(x)/ chunk_size))
# We don't want the 'chunk' and 'index' columns in the output
cols = [col for col in df.columns if col not in ['chunk']]
# groupby chunk and export each chunk to a different csv.
i = 0
for _, chunk in df.groupby('chunk'):
    chunk[cols].to_csv(f'/mnt/vdb/Rat/cd100/chunks/chunk{i}.csv', index=False)
    i += 1
print("complete")

complete


In [11]:
def createREPs(df, filename):
    _h_avg, h_final, c_final= get_reps(df['Sequence'].to_list())
    df.drop(columns=['Sequence'], inplace=True)
    df['reps']=_h_avg.tolist() # if there is a problem , might be here , possible solution is reindexing
    df.to_pickle(filename) # dont forget to change the file name to 0_613834

In [12]:
import glob
appended_reps = []
for infile in glob.glob("/mnt/vdb/Rat/cd100/reps/*.pkl"):
    # print(infile)
    appended_reps.append(infile)
print(len(appended_reps))

0


In [13]:
for infile in glob.glob("/mnt/vdb/Rat/cd100/chunks/*.csv"):
    #print("Read:"+infile)
    file_name = os.path.basename(infile)
    result="/mnt/vdb/Rat/cd100/reps/"+file_name.replace("csv", "pkl")
    if result in appended_reps :
        # print("found then skip : " , result)
        continue
    else:
        df = pd.read_csv(infile) 
        createREPs(df,result)
        print("Save:"+result)
        reset_device_memory()
        clear_jax_caches()
print("complete")

Save:/mnt/vdb/Rat/cd100/reps/chunk10.pkl
Save:/mnt/vdb/Rat/cd100/reps/chunk17.pkl
Save:/mnt/vdb/Rat/cd100/reps/chunk30.pkl
Save:/mnt/vdb/Rat/cd100/reps/chunk23.pkl
Save:/mnt/vdb/Rat/cd100/reps/chunk8.pkl
Save:/mnt/vdb/Rat/cd100/reps/chunk27.pkl
Save:/mnt/vdb/Rat/cd100/reps/chunk11.pkl
Save:/mnt/vdb/Rat/cd100/reps/chunk13.pkl
Save:/mnt/vdb/Rat/cd100/reps/chunk24.pkl
Save:/mnt/vdb/Rat/cd100/reps/chunk5.pkl
Save:/mnt/vdb/Rat/cd100/reps/chunk6.pkl
Save:/mnt/vdb/Rat/cd100/reps/chunk12.pkl
Save:/mnt/vdb/Rat/cd100/reps/chunk25.pkl
Save:/mnt/vdb/Rat/cd100/reps/chunk31.pkl
Save:/mnt/vdb/Rat/cd100/reps/chunk2.pkl
Save:/mnt/vdb/Rat/cd100/reps/chunk3.pkl
Save:/mnt/vdb/Rat/cd100/reps/chunk29.pkl
Save:/mnt/vdb/Rat/cd100/reps/chunk16.pkl
Save:/mnt/vdb/Rat/cd100/reps/chunk7.pkl
Save:/mnt/vdb/Rat/cd100/reps/chunk9.pkl
Save:/mnt/vdb/Rat/cd100/reps/chunk1.pkl
Save:/mnt/vdb/Rat/cd100/reps/chunk14.pkl
Save:/mnt/vdb/Rat/cd100/reps/chunk26.pkl
Save:/mnt/vdb/Rat/cd100/reps/chunk18.pkl
Save:/mnt/vdb/Rat/cd100/